자동으로 문자열에서 정보 뽑아내기

In [1]:
# Lab 12 Character Sequence RNN
import tensorflow as tf
###Disable Warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import logging
tf.get_logger().setLevel(logging.ERROR)
#import sys
#original_stdout = sys.stdout
#sys.stdout = None
#import warnings
#warnings.filterwarnings("ignore")

import numpy as np
tf.set_random_seed(777)  # reproducibility

sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> index
#enumerate(): 숫자를 부여해서 dict로 만들 때 사용하는 메소드
#dict는 key가 문자열임에 유의. idx2char와 char2idx는 서로 반대


#voc와 dic를 매우 간단하게 뽑아낼 수 있다.

# hyper parameters --> sample에 따라 자동으로 계산되도록 만든다.
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size #출력과 입력의 크기를 맞춰놓았다
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #) #맨 끝 샘플은 입력하지 않을거기 때문에 1을 뺀다.
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

#X, Y의 shape는 [batch_size, seq_length, input_dim]이어야 하지만, batch_size는 미지수로 비워놓고
#[None, seq_len]으로 취하고 input_dim은 뒤에서 one_hot encoding처리해도 똑같다.
X = tf.placeholder(tf.int32, [None, sequence_length])  # X data 
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True) #unit는 cell내부에 연결된 작은 node의 개념으로 생각해도 될 듯
initial_state = cell.zero_state(batch_size, tf.float32) #초기 state는 0으로 초기화
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)


# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size]) #열 개수만 hidden_size에 맞춰준다.
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
#fc를 이용해 X_for_fc가 입력되면 num_classes가 출력되는 레이어를 만든다. (activation function 없이)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length]) #가중치는 1로 초기화
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights) #이 logit에 쓰인 outputs는 BasicLSTMCell을 거쳤다. 즉, Activation function을 거쳤으므로 여기에 넣는 건 동작을 잘 하지 않는 원이이 될 수 있다고 함.
    # 이 예제는 동작을 잘 하는 것으로 보이므로 다른 예제가 필요할 듯
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))


'''
0 loss: 2.35377 Prediction: uuuuuuuuuuuuuuu
1 loss: 2.21383 Prediction: yy you y    you
2 loss: 2.04317 Prediction: yy yoo       ou
3 loss: 1.85869 Prediction: yy  ou      uou
4 loss: 1.65096 Prediction: yy you  a   you
5 loss: 1.40243 Prediction: yy you yan  you
6 loss: 1.12986 Prediction: yy you wann you
7 loss: 0.907699 Prediction: yy you want you
8 loss: 0.687401 Prediction: yf you want you
9 loss: 0.508868 Prediction: yf you want you
10 loss: 0.379423 Prediction: yf you want you
11 loss: 0.282956 Prediction: if you want you
12 loss: 0.208561 Prediction: if you want you #샘플 맨 앞에 공백을 주고 입력했으므로 그대로 나와야 맞다.
...
'''
pass

0 loss: 2.3335505 Prediction:           o    
1 loss: 2.1879745 Prediction:                
2 loss: 2.0537045 Prediction: yy  o     o  o 
3 loss: 1.9239314 Prediction: yy you   uo  ou
4 loss: 1.7358253 Prediction: yyyyou  ant  ou
5 loss: 1.5117447 Prediction: yyyyou  ant  ou
6 loss: 1.2620444 Prediction: yyyyou  ant you
7 loss: 1.0184368 Prediction: yfyyou  ant you
8 loss: 0.8032893 Prediction: yf you want you
9 loss: 0.62391526 Prediction: yf you want you
10 loss: 0.47817034 Prediction: yf you want you
11 loss: 0.36887828 Prediction: if you want you
12 loss: 0.2826835 Prediction: if you want you
13 loss: 0.21218924 Prediction: if you want you
14 loss: 0.15782769 Prediction: if you want you
15 loss: 0.115835555 Prediction: if you want you
16 loss: 0.08457473 Prediction: if you want you
17 loss: 0.060765598 Prediction: if you want you
18 loss: 0.04239126 Prediction: if you want you
19 loss: 0.029908229 Prediction: if you want you
20 loss: 0.0217493 Prediction: if you want you
21 loss: 0

긴 문자열 처리하기
- Window의 크기를 지정해서 convolution하듯이 처리한다

In [1]:
from __future__ import print_function

import tensorflow as tf
###Disable Warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import logging
tf.get_logger().setLevel(logging.ERROR)
#import sys
#original_stdout = sys.stdout
#sys.stdout = None
#import warnings
#warnings.filterwarnings("ignore")

import numpy as np
from tensorflow.contrib import rnn

tf.set_random_seed(777)  # reproducibility

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence)) #set()은 dictionary에서 key나 value를 뺀 것과 같다.
    #set()의 생성자는 입력받은 문자열을 모두 각각의 문자 세트(리스트처럼)로 만들어준다.
    #단, 중복은 제거된다!
char_dic = {w: i for i, w in enumerate(char_set)} 
    #set에게 enumerate를 취하면 dictionary를 만들 때 쓸 수 있다.
    #이 dict는 문자열(w)을 ket로 조회하여 알맞은 인덱스(i)를 찾을 때 쓰일 예정

#전부 dictionary화 시켜놓은 다음, batch를 구분할 예정

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number #구분할 구간의 길이
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length] #10개씩 뗀다
    y_str = sentence[i + 1: i + sequence_length + 1] #출력은 10개씩 뗀 바로 다음 구간(한칸씩 뒤로 취한 구간)
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index #char_dic을 이용하여 해당하는 인덱스 리스트를 만들어준다
    y = [char_dic[c] for c in y_str]  # y str to index #마찬가지

    dataX.append(x) #각각의 리스트를 dataX, dataY에 쌓아준다. 이러면 batch를 구분한 효과가 있음
    dataY.append(y)

batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape


# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

#Experiment: single Cell depth
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32) #초기 state는 0으로 초기화
outputs, _states = tf.nn.dynamic_rnn(
    #cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)
    cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)

#multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)
# outputs: unfolding size x hidden size, state = hidden size
#outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

'''
0 167 tttttttttt 3.23111
0 168 tttttttttt 3.23111
0 169 tttttttttt 3.23111
…
499 167  of the se 0.229616
499 168 tf the sea 0.229616
499 169   the sea. 0.229616

g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

'''
#맨 첫 문자 빼고는 모두 맞췄다.
#매끄럽지 못한 결과라고 하는데, 그 원인은 logit을 결정하는 부분과 네트웍의 깊이에 있다고 함.
#결과만 보면 한글자 빼고 잘 된거같다? BasicLSTM일때도 큰 문제가 안보인다
#loss가 천천히 줄어서 그런건가?
pass

teach th 0.2290337
459 122  thach the 0.2290337
459 123 toach them 0.2290337
459 124 hach them  0.2290337
459 125 rch them t 0.2290337
459 126 nh them to 0.2290337
459 127 t them to  0.2290337
459 128 ethem to l 0.2290337
459 129 toem ta lo 0.2290337
459 130 her ta lon 0.2290337
459 131 er ta long 0.2290337
459 132 r ta long  0.2290337
459 133  ta long f 0.2290337
459 134 to cong fo 0.2290337
459 135 h cong for 0.2290337
459 136 ncong for  0.2290337
459 137 tong for t 0.2290337
459 138 eng for th 0.2290337
459 139 n' for the 0.2290337
459 140 d for the  0.2290337
459 141 efor the e 0.2290337
459 142 tor the en 0.2290337
459 143  r the end 0.2290337
459 144 nkthe endl 0.2290337
459 145  the endle 0.2290337
459 146 toemendles 0.2290337
459 147 herendless 0.2290337
459 148 erendless  0.2290337
459 149 rendless i 0.2290337
459 150 tndless im 0.2290337
459 151 rdless imm 0.2290337
459 152 d ess imme 0.2290337
459 153  ess immen 0.2290337
459 154 ecs immens 0.2290337
459 155 rs immensi 0.229

In [4]:
set([1, 2, 3, 3])

{1, 2, 3}